In [1]:
import time
import torch
torch.set_num_threads(1)
from clebsch_gordan import get_real_clebsch_gordan, ClebschGordan
from sparse_accumulation_plain_torch import sparse_accumulation_loops, sparse_accumulation_index_add
import sparse_accumulation, sparse_accumulation_active_dim_first
import numpy as np

In [2]:
L_MAX = 8
clebsch = ClebschGordan(L_MAX).precomputed_
indices = get_real_clebsch_gordan(clebsch[L_MAX, L_MAX, L_MAX], L_MAX, L_MAX, L_MAX)

In [3]:
m1_aligned, m2_aligned = [], []
multipliers, mu_aligned = [], []
for mu in range(0, 2 * L_MAX + 1):
    for el in indices[mu]:
        m1, m2, multiplier = el
        m1_aligned.append(m1)
        m2_aligned.append(m2)
        multipliers.append(multiplier)
        mu_aligned.append(mu)
m1_aligned = torch.LongTensor(m1_aligned)
m2_aligned = torch.LongTensor(m2_aligned)
mu_aligned = torch.LongTensor(mu_aligned)
multipliers = torch.FloatTensor(multipliers)

In [ ]:
def sparse_accumulation_loops_active_dim_first(*args):
    return sparse_accumulation_loops(*args, active_dimension_first = True)

def sparse_accumulation_index_add_active_dim_first(*args):
    return sparse_accumulation_index_add(*args, active_dimension_first = True)

In [19]:
def benchmark_forward(BATCH_SIZE, N_FEATURES, active_dim, function, n_trials):
    if active_dim == 2:
        X1 = torch.randn(BATCH_SIZE, N_FEATURES, 2 * L_MAX + 1)
        X2 = torch.randn(BATCH_SIZE, N_FEATURES, 2 * L_MAX + 1)
    if active_dim == 0:
        X1 = torch.randn(2 * L_MAX + 1, BATCH_SIZE, N_FEATURES)
        X2 = torch.randn(2 * L_MAX + 1, BATCH_SIZE, N_FEATURES)
    if (active_dim != 0) and (active_dim != 2):
        raise ValueError("active dim can be only 0 or 2")
    times = []
    for _ in range(n_trials):
        begin = time.time()
        output = function(X1, X2, mu_aligned, 2 * L_MAX + 1, m1_aligned, m2_aligned, multipliers)
        times.append(time.time() - begin)
    return times


def benchmark_backward(BATCH_SIZE, N_FEATURES, active_dim, function, n_trials):
    if active_dim == 2:
        X1 = torch.randn(BATCH_SIZE, N_FEATURES, 2 * L_MAX + 1)
        X2 = torch.randn(BATCH_SIZE, N_FEATURES, 2 * L_MAX + 1)
    if active_dim == 0:
        X1 = torch.randn(2 * L_MAX + 1, BATCH_SIZE, N_FEATURES)
        X2 = torch.randn(2 * L_MAX + 1, BATCH_SIZE, N_FEATURES)
    if (active_dim != 0) and (active_dim != 2):
        raise ValueError("active dim can be only 0 or 2")
        
    X1.requires_grad = True
    X2.requires_grad = True
    times = []
    for _ in range(n_trials):
        begin = time.time()
        output = function(X1, X2, mu_aligned, 2 * L_MAX + 1, m1_aligned, m2_aligned, multipliers)
        output.backward(gradient=torch.ones_like(output))
        times.append(time.time() - begin)
    return np.array(times)

In [21]:
BATCH_SIZE = 1000
N_FEATURES = 100
times = benchmark_forward(BATCH_SIZE, N_FEATURES, 2, sparse_accumulation_loops, 10)
print("python loops; active dim 2; forward: ", np.mean(times[1:]))
times = benchmark_forward(BATCH_SIZE, N_FEATURES, 2, sparse_accumulation_index_add, 10)
print("torch index_add_; active dim 2; forward: ", np.mean(times[1:]))
times = benchmark_forward(BATCH_SIZE, N_FEATURES, 2, sparse_accumulation.SparseAccumulation.apply, 10)
print("cpp; active dim 2; forward: ", np.mean(times[1:]))

times = benchmark_forward(BATCH_SIZE, N_FEATURES, 0, sparse_accumulation_loops_active_dim_first, 10)
print("python loops; active dim 0; forward: ", np.mean(times[1:]))
times = benchmark_forward(BATCH_SIZE, N_FEATURES, 0, sparse_accumulation_index_add_active_dim_first, 10)
print("torch index_add_; active dim 0; forward: ", np.mean(times[1:]))
times = benchmark_forward(BATCH_SIZE, N_FEATURES, 0, sparse_accumulation_active_dim_first.SparseAccumulationActiveDimFirst.apply, 10)
print("cpp; active dim 0; forward: ", np.mean(times[1:]))




python loops; active dim 2; forward:  0.4338243272569444
torch index_add_; active dim 2; forward:  0.7381992075178359
cpp; active dim 2; forward:  0.0666847758822971
python loops; active dim 0; forward:  0.03847114245096842
torch index_add_; active dim 0; forward:  0.20851201481289333
cpp; active dim 0; forward:  0.026672919591267902


In [22]:
times = benchmark_backward(BATCH_SIZE, N_FEATURES, 2, sparse_accumulation_loops, 10)
print("python loops; active dim 2; backward: ", np.mean(times[1:]))
times = benchmark_backward(BATCH_SIZE, N_FEATURES, 2, sparse_accumulation_index_add, 10)
print("torch index_add_; active dim 2; backward: ", np.mean(times[1:]))
times = benchmark_backward(BATCH_SIZE, N_FEATURES, 2, sparse_accumulation.SparseAccumulation.apply, 10)
print("cpp; active dim 2; backward: ", np.mean(times[1:]))

times = benchmark_backward(BATCH_SIZE, N_FEATURES, 0, sparse_accumulation_loops_active_dim_first, 10)
print("python loops; active dim 0; backward: ", np.mean(times[1:]))
times = benchmark_backward(BATCH_SIZE, N_FEATURES, 0, sparse_accumulation_index_add_active_dim_first, 10)
print("torch index_add_; active dim 0; backward: ", np.mean(times[1:]))
times = benchmark_backward(BATCH_SIZE, N_FEATURES, 0, sparse_accumulation_active_dim_first.SparseAccumulationActiveDimFirst.apply, 10)
print("cpp; active dim 0; backward: ", np.mean(times[1:]))



python loops; active dim 2; backward:  5.021067725287543
torch index_add_; active dim 2; backward:  1.4698193868001301
cpp; active dim 2; backward:  0.1217292414771186
python loops; active dim 0; backward:  2.118829462263319
torch index_add_; active dim 0; backward:  0.4341705905066596
cpp; active dim 0; backward:  0.07336550288730198
